In [4]:
import math

# Physical constants
e = 1.602176634e-19      # Coulomb
h = 6.62607015e-34       # J*s
pi = math.pi
Z0 = 50.0                # Ohm

def Cd_from_EC_T1(EC_GHz, T1_s, f_q_GHz=5.0, Z0_ohm=50.0):
    """
    Compute coupling capacitor Cd from Purcell-limited decay rate:
        kappa_q = Z0 * omega_q^2 * (Cd^2 / Csum)
    Inputs:
        EC_GHz : transmon charging energy in GHz (EC/h)
        T1_s   : desired T1 in seconds
        f_q_GHz: qubit frequency in GHz (default 5 GHz)
        Z0_ohm : line impedance (default 50 Ohm)
    Returns:
        Cd in Farads and Csum in Farads
    """
    EC_Hz = EC_GHz * 1e9                   # EC/h in Hz
    Csum = e**2 / (2 * h * EC_Hz)          # EC/h = e^2/(2 h Csum)
    omega_q = 2 * math.pi * f_q_GHz * 1e9  # rad/s
    kappa_q = 1.0 / T1_s                   # 1/s
    Cd = math.sqrt(kappa_q * Csum / (Z0_ohm * omega_q**2))
    return Cd, Csum

# Example calculation with EC=0.16 GHz, T1=1 ms, fq=5 GHz
Cd_F, Csum_F = Cd_from_EC_T1(EC_GHz=0.85, T1_s=0.8e-3, f_q_GHz=4.5, Z0_ohm=50.0)
Cd_fF = Cd_F * 1e15
Csum_fF = Csum_F * 1e15

Cd_fF, Csum_fF


(0.026695345163083656, 22.788505087834263)

In [7]:
import numpy as np

def fluxonium_3node_myM(
    C_top, C_bot, C12, C1, C2, cg1=0.0, cg2=0.0,  # caps (same units throughout, e.g. F or fF)
    want_g=False, Z0=50.0
):
    """
    3-node two-floating-pads + drive, using YOUR transform:
        M = [[1,1,0],[1,-1,0],[0,0,1]]
        C_tilde = M^{-1} C M^{-1}
    Returns physical C_sum and Cg_eff; also EC from C_sum (Hz, GHz).
    If want_g=True, also returns a simple external-coupling g (Hz) from Cg_eff.
    """

    # ---- build 3x3 node-capacitance matrix in node order [x,y,z] ----
    Cg_drive = cg1 + cg2
    C = np.array([
        [C_top + C12 + C1,    -C12,             -C1],
        [-C12,                C_bot + C12 + C2, -C2],
        [-C1,                 -C2,              C1 + C2 + Cg_drive]
    ], dtype=float)

    # ---- your transform ----
    M = np.array([[1., 1., 0.],
                  [1.,-1., 0.],
                  [0., 0., 1.]], dtype=float)
    Minv = np.linalg.inv(M)
    C_tilde = Minv @ C @ Minv

    # In this basis, coordinates are q' = [(x+y)/2, (x-y)/2, z] = [c, d', z].
    # The physical differential coordinate is d = x - y = 2 d'.
    # Therefore: C_sum = 4 * C_tilde[d',d'],   Cg_eff = 2 * C_tilde[d',z].
    Cdd_scaled = C_tilde[1, 1]
    Cdz_scaled = C_tilde[1, 2]

    C_sum   = 1.0 * Cdd_scaled                 # physical qubit capacitance
    Cg_eff  = 1.0 * Cdz_scaled                 # effective drive-to-qubit cap (signed)

    # ---- EC from C_sum ----
    e  = 1.602176634e-19   # C
    h  = 6.62607015e-34    # J*s
    # If inputs were in fF, convert C_sum_fF -> F before computing EC.
    # Here we assume inputs are in SI F. If they are in fF, multiply C_sum by 1e-15 first.
    EC_Hz  = (e**2) / (2.0 * h * C_sum)        # Hz
    EC_GHz = EC_Hz / 1e9

    out = dict(C_sum=C_sum, Cg_eff=Cg_eff, EC_Hz=EC_Hz, EC_GHz=EC_GHz,
               C_tilde=C_tilde, C_matrix=C)

    if want_g:
        # very simple external coupling scale (harmonic-oscillator limit)
        # g ~ e/(2 sqrt(h Z0)) * (Cdz_scaled * 2) / C_sum  * w   (choose w externally for accuracy)
        # Here we only return the geometric prefactor G = e/(2 sqrt(h Z0)) * (Cg_eff / C_sum) (Hz / rad/s)
        G = (e / (2.0 * np.sqrt(h * Z0))) * (Cg_eff / C_sum)   # [Hz / sqrt(rad/s)] up to an ω factor
        out["g_geom"] = G
    return out

# ---------- example (numbers in fF) ----------
# If your numbers are in fF, convert once to SI before calling (or multiply C_sum by 1e-15 afterward).
fF = 1e-15
ex = fluxonium_3node_myM(
    C_top=25*fF, C_bot=44*fF, C12=3.2*fF,
    C1=0.27*fF, C2=0.05*fF, cg1=0.0, cg2=0.0
)
print("C_sum   =", ex["C_sum"]/fF, "fF")
print("Cg_eff  =", ex["Cg_eff"]/fF, "fF  (use abs for magnitude)")
print("EC      =", ex["EC_GHz"], "GHz")


C_sum   = 20.53 fF
Cg_eff  = -0.11000000000000001 fF  (use abs for magnitude)
EC      = 0.9435084912157389 GHz
